# 进阶数学问题
更复杂的数学问题如果单靠Instruct模型可能行不通，这跟人类一样，当我们在做高考数学题的时候，尤其是压轴题，我们读完题目总需要在草稿纸上不断的推演、不断的计算，我们并不能一气呵成的给出最准确的答案，也就是需要思考，模型同样也需要思考，而这正是推理型模型的优势所在，因为推理型模型经过大规模的训练，早已具备了反思和推理能力，无需我们添加CoT，模型就会通过长文本的思考给出最正确的答案。

推理型模型我们使用开源的Qwen3模型，该模型在数学问题上的表现非常好，我们可以直接使用该模型来解决数学问题。

> 之所以选择推理型模型而不是用CoT是因为更复杂的数学问题单纯靠CoT效果不一样会很好，因为更重要的是如果模型本身能力达不到，再怎么添加思考过程也没用。

In [1]:
### 加载模型：Qwen3-4B模型
from modelscope import AutoTokenizer, AutoModelForCausalLM

reasoning_model_name_or_path = '/home/lixinyu/weights/Qwen3-4B'  # 替换为你下载的模型路径
reasoning_tokenizer = AutoTokenizer.from_pretrained(reasoning_model_name_or_path)
reasoning_model = AutoModelForCausalLM.from_pretrained(reasoning_model_name_or_path,device_map='cuda:0', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.81it/s]


In [2]:
# Qwen3模型的推理函数
def resoning_inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手，请回答的尽量简洁",max_new_tokens=32768,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
    response = tokenizer.decode(output_ids,skip_special_tokens=True).strip("\n")

    return {
        "thinking_content":thinking_content,
        "content":content,
        "response":response
    }

In [3]:
prompt="""
早上8点，一列火车从A站出发，以每小时60英里的速度向东行驶。另一列火车上午9点从A站以东300英里的B站出发，以每小时75英里的速度向西行驶。这两列火车什么时候会合？
"""

In [4]:
print(resoning_inference(reasoning_model,reasoning_tokenizer,prompt)["response"])

<think>
好的，我现在要解决这个问题，两列火车什么时候会合。让我仔细读一遍题目，确保我理解正确。

题目说，早上8点，一列火车从A站出发，以每小时60英里的速度向东行驶。另一列火车上午9点从A站以东300英里的B站出发，以每小时75英里的速度向西行驶。问这两列火车什么时候会合。

首先，我需要明确两列火车的出发时间和方向。第一列火车从A站出发，时间是早上8点，向东行驶。第二列火车在上午9点从B站出发，向西行驶。B站位于A站东边300英里处。所以，当第二列火车出发时，第一列火车已经行驶了1小时，对吧？因为从8点到9点是1小时。

接下来，我需要计算两列火车在什么时候相遇。相遇的时候，它们的行驶距离之和应该等于A站和B站之间的300英里。不过，这里需要注意的是，第一列火车在第二列火车出发前已经行驶了一段时间，所以可能需要先计算这段时间内第一列火车行驶的距离，然后再加上之后两列火车的相对运动。

让我先画一个简单的示意图来帮助理解。A站和B站之间相距300英里，B站位于A站东边。第一列火车从A站出发向东，速度60mph。第二列火车从B站出发向西，速度75mph。第二列火车比第一列火车晚出发1小时，所以当第二列火车开始出发的时候，第一列火车已经走了60英里。因此，此时两列火车之间的距离是300英里减去60英里，也就是240英里。对吗？

不过，可能我的这个思路有问题。因为当第二列火车出发的时候，第一列火车已经向东行驶了1小时，所以它离A站的距离是60英里。而B站是在A站东边300英里，所以两列火车之间的初始距离应该是300英里减去第一列火车已经行驶的60英里，也就是240英里。这时候，两列火车相向而行吗？不，第一列火车向东，第二列火车向西，所以它们是相向而行，对吗？所以它们的相对速度是60+75=135mph。这样，它们相遇的时间应该是初始距离240英里除以相对速度135mph，得到的时间就是从第二列火车出发后的时间，也就是从9点开始算起的时间。

计算的话，240/135=1.777...小时，也就是1小时46.666...分钟，约等于1小时46分40秒。那么，相遇的时间就是9点加上1小时46分40秒，也就是10点46分40秒左右。不过，可能需要更精确的计算，或者用分数表示。

不过，可能还有另一种方法，考虑从第一列火车出发的时间开始计算。第一列火车从8点出发，第二